In [1]:
import cv2
import os
import numpy as np
import keras
import matplotlib.pyplot as plt
import download
from random import shuffle
from keras.applications import VGG16
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Activation
import sys
import h5py
import tensorflow as tf

In [3]:

def get_frames(current_dir, file_name, c):
    
    in_file = os.path.join(current_dir, file_name)
    
    images = []
    
    vidcap = cv2.VideoCapture(in_file)

    while(c):
        success,image = vidcap.read()
        c -= 1
        
    count = 0

    while count<_images_per_file:
                
        RGB_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        res = cv2.resize(RGB_img, dsize=(img_size, img_size),
                                 interpolation=cv2.INTER_CUBIC)
    
        images.append(res)
    
        success,image = vidcap.read()
    
        count += 1
        
    resul = np.array(images)
    
    resul = (resul / 255.).astype(np.float16)
        
    return resul

In [4]:
def get_transfer_values(current_dir, file_name):
    
    # Pre-allocate input-batch-array for images.
    shape = (_images_per_file,) + img_size_touple + (3,)
    
    image_batch = np.zeros(shape=shape, dtype=np.float16)
    
    image_batch = get_frames(current_dir, file_name)
      
    # Pre-allocate output-array for transfer-values.
    # Note that we use 16-bit floating-points to save memory.
    shape = (_images_per_file, transfer_values_size)
    transfer_values = np.zeros(shape=shape, dtype=np.float16)

    transfer_values =  image_model_transfer.predict(image_batch)
    #print(transfer_values.shape)
    return transfer_values

In [44]:
n = names_training[1]
l= labels_training[1]
print(l[1])

n = os.path.join(dir_fight,n)

print(n)
v = cv2.VideoCapture(n)
t = int(v.get(cv2.CAP_PROP_FRAME_COUNT))

t

1
/kaggle/input/violence-final/fight/v_Biking_g18_c06.avi


0

In [49]:
def proces_transfer(vid_names, labels):
    
    count = 0
    count_i = 0
    tam = len(vid_names)
    #print("tam =", tam)
    image_bitches = []
    # Pre-allocate input-batch-array for images.
    shape = (_images_per_file,) + img_size_touple + (3,)
    final_labels = []
    while count<tam:

        video_name = vid_names[count]
        label_n = labels[count]
        if label_n[0]==1:
            path = os.path.join(dir_fight,video_name)
        else:
            path = os.path.join(dir_not_fight, video_name)

        vidcap = cv2.VideoCapture(path)
        tam_i = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        #print(tam_i)  ---- correct till here

        if labels[count]==[0,1]:
            in_dir= dir_not_fight
        else:
            in_dir= dir_fight

        image_batches = []
        #print(shape)  2, 224, 224, 3

        while count_i<tam_i:
            image_batch = np.zeros(shape=shape, dtype=np.float16)
            image_batch = get_frames(in_dir, video_name, count_i)
            print(image_batch)
            count_i += 2
            image_batches.append(image_batch)

        image_bitches.append(image_batches)
        image_bitches = np.array(image_bitches)
        print(image_bitches.shape) #wrong shape 
         # Note that we use 16-bit floating-points to save memory.
        shape = (image_bitches.shape[0]*image_bitches.shape[1], image_bitches.shape[2], transfer_values_size)
        transfer_values = np.zeros(shape=shape, dtype=np.float16)
        
        transfer_values =  image_model_transfer.predict(image_batch)
         
        labels1 = labels[count]
        
        aux = np.ones([image_bitches.shape[1],2])
        
        labelss = labels1*aux
        final_labels.append(labelss)
    
        ##############################################################
        #yield transfer_values, labelss
        ##############################################################
        count+=1
    final_labels = final_labels.reshape([final_labels[0]*final_labels[1], final_labels[2]])
    return final_labels, image_bitches

In [6]:
import random


dir_fight="/kaggle/input/violence-final/fight"
dir_not_fight="/kaggle/input/violence-final/not_fight"
list_fight=os.listdir(dir_fight)
list_no_fight=os.listdir(dir_not_fight)


fight_final=random.sample(list_fight, 800)
no_fight_final=random.sample(list_no_fight,800)


fight_labels = []
no_fight_labels = []
for i in range (800):
    fight_labels.append([1,0])
    no_fight_labels.append([0,1])



final = fight_final + no_fight_final
labels = fight_labels + no_fight_labels

In [7]:
c = list(zip(final,labels))
shuffle(c)
    
names, labels = zip(*c)

In [8]:

# Frame size  
img_size = 224

img_size_touple = (img_size, img_size)

# Number of channels (RGB)
num_channels = 3

# Flat frame size
img_size_flat = img_size * img_size * num_channels

# Number of classes for classification (Violence-No Violence)
num_classes = 2

# Number of files to train
_num_files_train = 1

# Number of frames per video
_images_per_file = 2

# Number of frames per training set
_num_images_train = _num_files_train * _images_per_file

image_model = VGG16(include_top=True, weights='imagenet')

553467904/553467096 [==============================] - 7s 0us/step


In [9]:
input_shape = image_model.layers[0].output_shape[1:3]
print(input_shape)

[]


In [10]:


# We will use the output of the layer prior to the final
# classification-layer which is named fc2. This is a fully-connected (or dense) layer.
transfer_layer = image_model.get_layer('fc2')

image_model_transfer = Model(inputs=image_model.input,
                             outputs=transfer_layer.output)

transfer_values_size = K.int_shape(transfer_layer.output)[1]


#print("The input of the VGG16 net have dimensions:",K.int_shape(image_model.input)[1:3])

#print("The output of the selecter layer of VGG16 net have dimensions: ", transfer_values_size)

In [24]:



training_set = int(len(names)*0.8)
test_set = int(len(names)*0.2)

names_training = names[0:training_set]
names_test = names[training_set:]

labels_training = labels[0:training_set]
labels_test = labels[training_set:]

#correct


In [50]:

frames, labels = proces_transfer(names_training, labels_training)
frames_t, labels_t = proces_transfer(names_test, labels_test)
frames = frames.reshape([frames.shape[1], frames.shape[0], frames.shape[2], frames.shape[3], frames.shape[4]])
frames_t = frames_t.reshape([frames_t.shape[1], frames_t.shape[0], frames_t.shape[2], frames_t.shape[3], frames_t.shape[4]])

UnboundLocalError: local variable 'image' referenced before assignment

In [ ]:
chunk_size = 4096
#n_chunks = 20
rnn_size = 512

In [ ]:
model1 = Sequential()
input_1 = tf.keras.layers.Input(shape=(1, 4096))
conv2d_1 = tf.keras.layers.Conv2D(32, kernel_size=3, activation=tf.keras.activations.relu)(input_1)
pool1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None, **kwargs)(conv2d_1)

In [ ]:



model2 = Sequential()
input_2 = tf.keras.layers.Input(shape=(1, 4096))
conv2d_2 = tf.keras.layers.Conv2D(32, kernel_size = 3, activation=tf.keras.activations.relu)(input_2)
pool2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None, **kwargs)(conv2d_2)

In [ ]:


model = keras.layers.concatenate([model1, model2])
FC1 = Dense(1024, activation='relu')(model)
lstm_layer = LSTM(rnn_size)(FC1)
output = tf.keras.layers.Dense(units=2,
                               activation=tf.keras.activations.softmax)(lstm_layer)

In [ ]:

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
full_model = tf.keras.Model(inputs = [model1, model2], outputs=[output])
print(full_model.summary())

In [ ]:

epoch = 200
batchS = 500

history = full_model.fit(inputs = [frames[1], frames[2]], outputs = [labels], epochs=epoch,
                    validation_data=([frames_t[1],frames_t[2]], [labels_t]), 
                    batch_size=batchS, verbose=2)

In [ ]:
result = full_model.evaluate(np.array(data_test), np.array(target_test))


In [ ]:

for name, value in zip(model.metrics_names, result):
    print(name, value)

In [ ]:



out_dir = '/kaggle/working/'

model_json = full_model.to_json()
with open(os.path.join(out_dir,'model1.json'), "w") as json_file:
    json_file.write(model_json)

# SAVE MODEL WEIGHTS
full_model.save_weights(os.path.join(out_dir,'model1.h5'))
full_model.save_weights(os.path.join('model1.h5'))
